In [43]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import re
import nltk
from sklearn.metrics import confusion_matrix,accuracy_score
import joblib
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [44]:
df=pd.read_csv("amazon_data3.csv")
df.dropna(subset=['review_text','review_rating'],axis=0,inplace=True)

In [45]:
df1=df[['review_text','review_rating']]
df1['Sentiment']=np.where(df1['review_rating']>3,1,0)

In [46]:
# df1[df1['Sentiment']==0]
df1.dropna(axis=0,inplace=True)
df1.reset_index(drop=True,inplace=True)

In [47]:
dataset=df1[['review_text']]

In [48]:
dataset.head()

,review_text
0,Poor battery lifePoor build quality
1,Work and study
2,Defective loptop
3,Nice laptopSpeed also goodOver looking superBa...
4,Laptop is good and smooth it's working nicely ...


### Data Cleaning

In [49]:

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\INMAANN2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [50]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [51]:
ps=PorterStemmer()
all_stopwords=stopwords.words('english')
all_stopwords.remove('not')

In [52]:
corpus=[]
for i in range(0,df1.shape[0]):
#     print(i)
    review=re.sub('[^a-zA-Z]',' ',dataset['review_text'][i])
    review=review.lower()
    review=review.split()
    review=[ps.stem(word) for word in review if not word in set(all_stopwords)]
    review=' '.join(review)
    corpus.append(review)

In [53]:
cvFile='c1_BoW_Sentiment_Model.pkl'
cv=pickle.load(open(cvFile,'rb'))

In [54]:
X_fresh=cv.transform(corpus).toarray()
X_fresh.shape

(389, 1393)

In [55]:
classifier=joblib.load('c2_Classifer_Sentiment_Model')

In [56]:
y_pred=classifier.predict(X_fresh)
print(y_pred)

[1 1 0 1 1 1 1 0 0 1 1 0 1 1 1 0 1 1 1 1 0 0 1 1 0 1 1 1 1 0 0 1 1 0 1 1 1
 1 0 0 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1
 0 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
 0 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 0 1 0 1
 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 0 1 1 1 0 1 1 1 1 0 1 1 1
 0 1 1 1 1 1 0 0 1 0 1 1 1 1 1 1 0 0 1 0 1 1 1 1 1 1 0 0 1 0 1 1 1 0 1 1 1
 0 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 0 1 1 1 0 1 0 1 1 0 0 1 1 1
 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 1 1 0 1 1 1
 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [57]:
dataset['predicted_label']=y_pred.tolist()

In [58]:
dataset.head()

,review_text,predicted_label
0,Poor battery lifePoor build quality,1
1,Work and study,1
2,Defective loptop,0
3,Nice laptopSpeed also goodOver looking superBa...,1
4,Laptop is good and smooth it's working nicely ...,1


In [65]:
# dataset.to_csv("Predicted.csv")

In [ ]:
dataset['Actual']=df1['Sentiment']

In [ ]:
cm=confusion_matrix(df1['Sentiment'],y_pred)

In [60]:
print(cm)
accuracy_score(df1['Sentiment'],y_pred)

[[ 47  37]
 [ 23 282]]


0.8457583547557841

In [61]:
df['Predicted_label']=y_pred

In [62]:
positive_reviews=df[df['Predicted_label']==1]

In [63]:
positive_df=pd.DataFrame(positive_reviews.groupby(['name','price','asin'])['Predicted_label'].count())

In [71]:
positive_df.reset_index(inplace=True)

In [78]:
final_laptop=positive_df['name'].iloc[np.argmax(positive_df['Predicted_label'])]

In [79]:
print(f'Based on the number of positive reviews, the laptop which got most of the positive review is {final_laptop}')

Based on the number of positive reviews, the laptop which got most of the positive review is         HP Victus Gaming Laptop AMD Ryzen 5 5600H,15.6inch(39.6cm) FHD Gaming Laptop/16GB RAM/512GB SSD/9ms Response Time/144Hz/AMD Radeon RX 6500M Graphics/Backlit KB/Win11/B&O/Xbox Pass 15-fb0134AX       
